In [69]:
import pandas as pd
import numpy as np
import requests
import json

### 1. We start off by getting the matches (DO NOT RUN THIS UNLESS YOU ARE SURE)

In [100]:
### get_matches returns a dataframe from the opendota API with given parameters
### don't run this too often, as there is a request limit of one per second and 50000 pr. month
def get_matches(matches=10000, start_time_less_than=1594771200):
    # we build the base url for the opendota api 
    base = 'https://api.opendota.com/api/explorer'
    # we create the SQL percent encoded query. The query is as follows
    #"""
    #SELECT *
    #FROM public_matches
    #LEFT JOIN public_player_matches
    #ON public_matches.match_id = public_player_matches.match_id
    #WHERE lobby_type=7 AND game_mode=22 AND avg_mmr!=0 AND start_time<1594771200
    #ORDER BY start_time DESC
    #LIMIT 1000000
    #"""
    matches_request = f'?sql=SELECT%20*%0AFROM%20public_matches%0ALEFT%20JOIN%20public_player_matches%0AON%20public_matches.match_id%20%3D%20public_player_matches.match_id%0AWHERE%20lobby_type%3D7%20AND%20game_mode%3D22%20AND%20avg_mmr!%3D0%20AND%20start_time%3C{start_time_less_than}%0AORDER%20BY%20start_time%20DESC%0Alimit%20{str(matches)}'

    url = base + matches_request

    result = requests.get(url)
    print(result)
    j_matches = result.json()

    return pd.DataFrame(j_matches["rows"])

In [101]:
df_matches_response = get_matches(matches=500000, start_time_less_than=1594771200)

<Response [200]>


### 2. Let's start inspecting our dataframe

**We do have to be mindful that all matches are duplicated 10 times, since there is a row for each picked hero in each match**

In [126]:
df_matches = df_matches_response
df_matches

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,...,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5
0,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,5511875905,4621658882,True,1594676135,1853,3379,1,7,22,39,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
499996,5511875905,4621658882,True,1594676135,1853,3379,1,7,22,39,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
499997,5511875905,4621658882,True,1594676135,1853,3379,1,7,22,39,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499998,5511875905,4621658882,True,1594676135,1853,3379,1,7,22,39,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**As seen below, all the datatypes seem to come in the right format**

In [127]:
df_matches.dtypes

match_id           int64
match_seq_num      int64
radiant_win         bool
start_time         int64
duration           int64
avg_mmr            int64
num_mmr            int64
lobby_type         int64
game_mode          int64
avg_rank_tier      int64
num_rank_tier      int64
cluster            int64
player_slot        int64
hero_id            int64
radiant_1        float64
radiant_2        float64
radiant_3        float64
radiant_4        float64
radiant_5        float64
dire_1           float64
dire_2           float64
dire_3           float64
dire_4           float64
dire_5           float64
dtype: object

**There are zero null values, as this was handled in the SQL query to the API**

In [128]:
df_matches.isna().sum()

match_id              0
match_seq_num         0
radiant_win           0
start_time            0
duration              0
avg_mmr               0
num_mmr               0
lobby_type            0
game_mode             0
avg_rank_tier         0
num_rank_tier         0
cluster               0
player_slot           0
hero_id               0
radiant_1        450000
radiant_2        450000
radiant_3        450000
radiant_4        450000
radiant_5        450000
dire_1           450000
dire_2           450000
dire_3           450000
dire_4           450000
dire_5           450000
dtype: int64

**Let's have a look at the "Player Slot" column**

* 0-4: Radiant team
* 128-132: Dire team

If the player slot is between 0-4 the player is on the radiant team and if the player is in 128-132 the player is on the dire team

In [129]:
df_matches["player_slot"].value_counts()

132    50000
131    50000
130    50000
129    50000
128    50000
4      50000
3      50000
2      50000
1      50000
0      50000
Name: player_slot, dtype: int64

**Let's reformat that to "Radiant" and "Dire"**

We do not care what slot on dire the player was located in. Therefore we simply convert it to "Radiant" and "Dire"

In [130]:
slot_dict = {0: "radiant_1",
             1: "radiant_2",
             2: "radiant_3",
             3: "radiant_4",
             4: "radiant_5",
             128: "dire_1",
             129: "dire_2",
             130: "dire_3",
             131: "dire_4",
             132: "dire_5"}

In [131]:
{slot_dict[i] for i in slot_dict}

{'dire_1',
 'dire_2',
 'dire_3',
 'dire_4',
 'dire_5',
 'radiant_1',
 'radiant_2',
 'radiant_3',
 'radiant_4',
 'radiant_5'}

In [134]:
for slot in slot_dict:
    df_matches.loc[df_matches['player_slot'] == slot, slot_dict[slot]] = 1
df_matches.head(20)

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,...,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5
0,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
5,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
6,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
df_matches["player_slot"].value_counts()

132    50000
131    50000
130    50000
129    50000
128    50000
4      50000
3      50000
2      50000
1      50000
0      50000
Name: player_slot, dtype: int64

Since there is an equal amount of heroes on each side, this means that we split it correctly

In [80]:
df_matches.rename(columns={"player_slot": "team"})

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,team,hero_id
0,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,6,154,Dire,84
1,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,6,154,Dire,8
2,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,6,154,Dire,39
3,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,6,154,Dire,29
4,5513437212,4623088705,True,1594771194,2467,3265,4,7,22,61,6,154,Dire,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,5511875905,4621658882,True,1594676135,1853,3379,1,7,22,39,4,183,Radiant,29
499996,5511875905,4621658882,True,1594676135,1853,3379,1,7,22,39,4,183,Radiant,36
499997,5511875905,4621658882,True,1594676135,1853,3379,1,7,22,39,4,183,Radiant,21
499998,5511875905,4621658882,True,1594676135,1853,3379,1,7,22,39,4,183,Radiant,56


**We do not care about `[num_mmr, lobby_type, game_mode, avg_rank_tier]`, so these columns are dropped**

In [86]:
df_matches.drop(["num_mmr", "lobby_type", "game_mode", "avg_rank_tier"], axis=1, inplace=True)

In [88]:
df_matches.head(10)

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_rank_tier,cluster,player_slot,hero_id
0,5513437212,4623088705,True,1594771194,2467,3265,6,154,Dire,84
1,5513437212,4623088705,True,1594771194,2467,3265,6,154,Dire,8
2,5513437212,4623088705,True,1594771194,2467,3265,6,154,Dire,39
3,5513437212,4623088705,True,1594771194,2467,3265,6,154,Dire,29
4,5513437212,4623088705,True,1594771194,2467,3265,6,154,Dire,80
5,5513437212,4623088705,True,1594771194,2467,3265,6,154,Radiant,46
6,5513437212,4623088705,True,1594771194,2467,3265,6,154,Radiant,75
7,5513437212,4623088705,True,1594771194,2467,3265,6,154,Radiant,96
8,5513437212,4623088705,True,1594771194,2467,3265,6,154,Radiant,4
9,5513437212,4623088705,True,1594771194,2467,3265,6,154,Radiant,25


**We want to go from this table:**

| match_id | ... | player_slot | hero_id |
|------|------|------|------|
|5513437212|...|Dire|84|
|5513437212|...|Dire|8|
|5513437212|...|Dire|39|
|5513437212|...|Dire|29|
|5513437212|...|Dire|80|
|5513437212|...|Radiant|46|
|5513437212|...|Radiant|75|
|5513437212|...|Radiant|96|
|5513437212|...|Radiant|4|
|5513437212|...|Radiant|25|

**To this table:**

| match_id | ... | radiant1 | radiant2 | radiant3 | radiant4 | radiant5 |  dire1 | dire2 | dire3 | dire4 | dire5 |
|------|------|------|------|------|------|------|------|------|------|------|------|
|5513437212|...|46|75|96|4|25|84|8|39|29|80|